In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import random as rd

In [ ]:
'''
Début de code propre pour filtrage non-linéaire avec MLP. N'aboutira pas pour cause de reshape, un autre code sera adopté pour répondre à
cette problématique
'''

In [3]:
# Generate low-passed random signal
def generate_randx(duration, nb_val):

    """ Generate low-passed random signal with real and imaginary part.\n
    Parameters : \n
    \t duration : time length of signal,\n
    \t nb_val : number of samples in duration """

    x_axis = np.linspace(0, duration-1, nb_val)
    x_data = np.zeros(x_axis.size, dtype = np.complex128)
    for i in range(x_axis.size):
        x_data[i] = rd.randint(0,100) + rd.randint(0,100)*1j
    x_data = np.convolve(x_data, np.array([1,1,1,1,1,1,1,1,1,1,1,1,1])/13, mode='same')
    return x_axis, x_data

    '''
    Il nous faut le x_axis, pour montrer la réponse du truc 
    ET
    le x_data pour les prédictions
    '''

In [8]:
#Decal ne fonctionne pas, on préfèrera memory encore en-dessous

def decal(x,h):
    '''
    Va associer à chaque tuple de h.shape entrées sa sortie
    Doit prendre en compte le décalage dû à l'absence des retards
    TODO: Prendre en compte les complexes!!!
    '''
    #x_c=np.copy(x)
    #y_c=np.copy(y)
    hlen=len(h)
    xlen=len(x)
    
    #Liste des y sans les trucs en trop
    #y_d=y_d[:-(len(h))+1]
    #Rendu inutile avec mode='same' dans np.convolve

    #Liste des x rassemblés en len(h)
    i=0
    x_d=np.array([])
    for i in range(xlen):
        x_temp_re=np.array([])
        x_temp_im=np.array([])
        for j in range(i,hlen+i):
            if (j>(xlen-1)):
                x_array_value=[0]
                x_temp_re=np.append(x_temp_re,x_array_value)
                x_temp_im=np.append(x_temp_im,x_array_value)
            else:
                x_array_value=x[j]
                x_temp_re=np.append(x_temp_re,x_array_value)
                x_temp_im=np.append(x_temp_im,x_array_value)
        x_d=np.append(x_d,np.concatenate(x_temp_re,x_temp_im))
    return x_d

#return(np.reshape(x_d,(-1,hlen)))

def memory(x, mem_depth):

    """ Returns list of arrays of size 2*mem_depth containing real and imaginary parts of last samples (memory).\n
    Parameters : \n
    \t x : input signal,\n
    \t mem_depth : number of last samples taken into account in output at instant t """
    final = []
    for i in range(mem_depth, x.size+1):
        sample = x[i-mem_depth:i]
        x_r = np.zeros(mem_depth) # real parts of sample
        x_i = np.zeros(mem_depth) # imaginary parts of sample
        for k in range(sample.size): # fill in real and imaginary samples
            x_r[k] = sample[k].real
            x_i[k] = sample[k].imag
        final.append(np.concatenate((x_r, x_i)))
        final.reshape((1,-1))
    return final


In [9]:
def output(x,h1,h2):
    '''
    On calcule la sortie, là on a que deux h, mais réfléchir à comment iplémenter plus (un tableau de h pour chaque ordre etc...)
    On sépare Im et Re
    '''
    x_squared = x**2
    sortie = np.convolve(x,h1,mode='same') + np.convolve(x_squared,h2,mode='same')
    taille = len(sortie)
    sortie_finale = np.zeros(taille,2)
    for k in range(taille):
        sortie_finale[k][0] = val.real
        sortie_finale[k][1] = val.imag
    return sortie_finale

In [10]:
#Commencer par générer une entrée, avec randx, et les convolutions
x_axe, x_samples = generate_randx(10,50)
h1 = np.array([1,2,3,5])
h2 = np.array([3,1,8,1])
memory_size = len(h1)
#TODO: fonction d'erreur si les deux convolutions ne sont pas de même taille

#On crée le bon tableau au bon format
input_mlp = memory(x_samples,h1)
#input_mlp = np.array(input_mlp)

#NORMALISATION
scaler = StandardScaler()
input_mlp = scaler.fit_transform(input_mlp)
#print(input_mlp)

#Puis, on calcule sa sortie
y = output(x_samples,h1,h2)
#y = y.reshape(-1,1)
#print(y)
scaler = StandardScaler()
y = scaler.fit_transform(y)
#print(y)

TypeError: only integer scalar arrays can be converted to a scalar index

In [21]:
u = 1+1j
print(u)
print(u.real)
print(u.imag)

(1+1j)
1.0
1.0
